# Test GPU Training
I was getting the same error here: https://www.reddit.com/r/tensorflow/comments/dxnnq2/i_am_getting_an_error_while_running_the_rnn_lstm/f7wv17k/, and used the example code from the thread to debug.

### Import Tensorflow and Check GPU Support

In [1]:
import tensorflow as tf
print('Built with Cuda: {}'.format(tf.test.is_built_with_cuda()))
print('Build with GPU support: {}'.format(tf.test.is_built_with_gpu_support()))
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Built with Cuda: True
Build with GPU support: True
Num GPUs Available:  1


### Load Dat, Build Model, Train MNIST

In [ ]:
batch_size = 64
input_dim = 28
units = 64
output_size = 10

def build_model(allow_cudnn_kernel=True):
    if allow_cudnn_kernel:
        lstm_layer = tf.keras.layers.LSTM(units, input_shape=(None, input_dim))
    else:
        lstm_layer = tf.keras.layers.RNN(tf.keras.layers.LSTMCell(units),
                                         input_shape=(None, input_dim))
    model = tf.keras.models.Sequential([
        lstm_layer,
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(output_size, activation='softmax')]
    )
    return model

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

sample, sample_label = x_train[0], y_train[0]
model = build_model(allow_cudnn_kernel=True)

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='sgd',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          batch_size=batch_size,
          epochs=5)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 8s 135us/sample - loss: 0.9661 - accuracy: 0.6939 - val_loss: 0.5382 - val_accuracy: 0.8334
Epoch 2/5
60000/60000 [==============================] - 5s 80us/sample - loss: 0.4097 - accuracy: 0.8764 - val_loss: 0.3104 - val_accuracy: 0.9009
Epoch 3/5
60000/60000 [==============================] - 5s 82us/sample - loss: 0.2664 - accuracy: 0.9198 - val_loss: 0.2749 - val_accuracy: 0.9137
Epoch 4/5
60000/60000 [==============================] - 5s 79us/sample - loss: 0.2080 - accuracy: 0.9382 - val_loss: 0.2056 - val_accuracy: 0.9327
Epoch 5/5
34176/60000 [================>.............] - ETA: 1s - loss: 0.1810 - accuracy: 0.9450